# Task 1 : CSV Import Mastery
- **Objektive:** Use the code template for importing data from a CSV file into TinyFlux
- **Instructions:**
    1. Take the provided CSV import template and import a new CSV file 
    2. Modify the template to add a new field and tag (e.g., add a "device_type"tag)
    3. Vertify that the data has been correctly inserted by querying the first 10 records.

- **Goal:** Understand how to manipulate CSV data and map fields and tags when importing into TinyFlux.

In [2]:
# Libraries
import pandas as pd
import csv
from tinyflux import TinyFlux, Point, FieldQuery, TagQuery, TimeQuery
from datetime import datetime, timezone, timedelta

# Settings
import warnings
warnings.filterwarnings("ignore")

# Abkürzungen für Query-Typen
time = TimeQuery()
tags = TagQuery()
field = FieldQuery()

In [3]:
# Define the function to transform the list of Point objects into a DataFrame
def points_to_dataframe(points_list):
    data = []
    
    # Extract the relevant information
    for point in points_list:
        data.append({
            "time": point.time,  # Access time attribute
            "sensor_id": point.tags['sensor_id'],  # Access sender_id from tags dictionary
            "temperature": point.fields['temperature'],  # Access receiver_id from tags dictionary
            "status": point.tags['status']  # Access status from tags dictionary
        })

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Convert time to datetime
    df['time'] = pd.to_datetime(df['time'])

    return df

In [4]:
# Erstellung der Tinyflux-Datenbank (wird bereitgestellt)
db = TinyFlux("temperatures2.db")

In [5]:
# CSV-Datei in Dataframe laden
df = pd.read_csv('example_data/synthetic_temperature_readings.csv')

# Das Format des Zeitstempels in Pandas setzen
df['time']= pd.to_datetime(df['time'], format='mixed')

# Erste 10 Records des Dataframes anzeigen
df.head(10)

,time,sensor_id,temperature,status
0,2023-01-01 00:00:00,52,20.377761,operational
1,2023-01-01 08:44:41,93,2.217658,maintenance
2,2023-01-01 17:29:22,15,14.912385,offline
3,2023-01-02 02:14:03,72,6.517424,operational
4,2023-01-02 10:58:44,61,36.684591,operational
5,2023-01-02 19:43:25,21,-9.623282,operational
6,2023-01-03 04:28:06,83,1.266640,operational
7,2023-01-03 13:12:47,87,8.267841,operational
8,2023-01-03 21:57:28,75,14.390490,operational
9,2023-01-04 06:42:09,75,32.540876,operational


# Task 2: Manual Data Entry Practice
- **Objective**: Manually enter time-series records into TinyFlux.
- **Instrusctions:**
    1. Using the manual data entry template, add 5 new records manuall (e.g. representing temperature readings over 5 minutes)
    2. Eacht record should habe a temperature field, location tag, and time fields. Choose appropriate values for these fields.
    3. After inserting the records, query, and print them to ensure they were entered correctly.

- **Goal:** Practice creating individual records manually and inserting them into the TinyFlux database.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   time         1000 non-null   datetime64[ns]
 1   sensor_id    1000 non-null   int64         
 2   temperature  1000 non-null   float64       
 3   status       1000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 31.4+ KB


In [7]:
# Deskriptive Statistiken des Dataframes
df.describe(include='all')

,time,sensor_id,temperature,status
count,1000,1000.000000,1000.000000,1000
unique,NaN,NaN,NaN,3
top,NaN,NaN,NaN,operational
freq,NaN,NaN,NaN,858
mean,2023-07-01 23:59:59.513999872,49.560000,15.392825,NaN
min,2023-01-01 00:00:00,1.000000,-9.988124,NaN
25%,2023-04-01 23:59:59.750000128,24.000000,2.945454,NaN
50%,2023-07-01 23:59:59.500000,50.000000,16.125006,NaN
75%,2023-09-30 23:59:59.249999872,75.000000,27.474836,NaN
max,2023-12-31 00:00:00,99.000000,39.967675,NaN


In [8]:
# Read CSV and insert data into TinyFlux
with open("example_data/synthetic_temperature_readings.csv", mode="r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        point = Point(
            time=datetime.strptime(row["time"], "%Y-%m-%d %H:%M:%S"),
            measurement="blockchain_transactions",
            fields={
                "temperature": float(row["temperature"])
                },  # Only numeric fields
            tags={
                "sensor_id": row["sensor_id"],
                "status": row["status"]  # Move non-numeric fields to tags
            }
        )
        db.insert(point)

In [9]:
# Create manual records
p1 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
    tags={"sensor_id": "20", "status": "operational"},
    fields={"temperature": 15.240971}
)

p2 = Point(
    time=datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc),
    tags={"sensor_id": "30", "status": "maintenance"},
    fields={"temperature": 20.517200}
)

# Insert into the DB.
db.insert_multiple([p1, p2])

2

In [10]:
# Definieren der Query
time_query_conditions = (time >= datetime(2024, 1, 1, 0, 0, 0, tzinfo=timezone.utc))

# Datenabfrage mittels der definierten Query
my_results = db.search(time_query_conditions)

# Erstellung eines Dataframes aus den Query-Resultaten
# Die ugehörige Funktion "points_to_dataframe()" wurde zu Beginn dieses Notebooks in den Arbeitsspeicher geladen
df_result_time = points_to_dataframe(my_results)
df_result_time

,time,sensor_id,temperature,status
0,2024-01-01 00:00:00+00:00,20,15.240971,operational
1,2024-01-01 00:00:00+00:00,30,20.517200,maintenance


In [11]:
df_result_time.describe(include='all')

,time,sensor_id,temperature,status
count,2,2,2.000000,2
unique,NaN,2,NaN,2
top,NaN,20,NaN,operational
freq,NaN,1,NaN,1
mean,2024-01-01 00:00:00+00:00,NaN,17.879085,NaN
min,2024-01-01 00:00:00+00:00,NaN,15.240971,NaN
25%,2024-01-01 00:00:00+00:00,NaN,16.560028,NaN
50%,2024-01-01 00:00:00+00:00,NaN,17.879085,NaN
75%,2024-01-01 00:00:00+00:00,NaN,19.198143,NaN
max,2024-01-01 00:00:00+00:00,NaN,20.517200,NaN


# Task3: Query by Time and Analyze



# Task 4: